In [6]:
import json

import pandas as pd
from numpy.array_api import result_type

dataset_dir = "./dataset"

with open('crowd_source.json') as f:
    crowd_data = json.load(f)

In [8]:
crowd_data

{'a happy event': [['cast member',
   'theis schmidt',
   'cast member',
   'erika sainte',
   0,
   5,
   -0.013]],
 'a night at the opera': [['cast member',
   'jude law',
   'cast member',
   'rolfe sedan',
   1,
   4,
   -0.013]],
 'william hurt': [['cast member',
   'behind the candelabra',
   'cast member',
   'the blue butterfly',
   2,
   3,
   0.152]],
 'frederick worlock': [['cast member',
   'bloody mama',
   'cast member',
   'airport',
   2,
   3,
   0.152]],
 'cho geunhyeon': [['country of citizenship',
   'angola',
   'country of citizenship',
   'south korea',
   2,
   3,
   -0.013]],
 'conan the barbarian': [['voice actor',
   'rose mcgowan',
   'cast member',
   'rose mcgowan',
   2,
   3,
   0.152]],
 'eskimo': [['production designer',
   'john lee mahin',
   'screenwriter',
   'john lee mahin',
   1,
   4,
   -0.013]],
 'free willy 3 the rescue': [['screenwriter',
   'angel de andres miquel',
   'screenwriter',
   'keith walker',
   2,
   3,
   -0.013]],
 'guardians

In [9]:
with open("./exports/crowd_data.tsv", "r") as f:
    crowd_df = pd.read_csv(f, sep="\t", index_col="AssignmentId")
    crowd_df = crowd_df.sort_values(by=["HITId", "HITTypeId", "WorkerId"])

crowd_df

,HITId,HITTypeId,Title,Reward,WorkerId,AssignmentStatus,WorkTimeInSeconds,LifetimeApprovalRate,Input1ID,Input2ID,Input3ID,AnswerID,AnswerLabel,FixPosition,FixValue
AssignmentId,,,,,,,,,,,,,,,
4,1,7QT,Is this triple correct or incorrect?,$0.50,1726JMZQW,Submitted,80,70%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
1,1,7QT,Is this triple correct or incorrect?,$0.50,2133ICYWE97,Submitted,60,99%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
2,1,7QT,Is this triple correct or incorrect?,$0.50,2133U7HKDLO,Submitted,40,40%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,yes,yes
5,1,7QT,Is this triple correct or incorrect?,$0.50,2134U7HKDMM,Submitted,2,70%,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
3,1,7QT,Is this triple correct or incorrect?,$0.50,928UJANWZ12,Submitted,50,98%,wd:Q11621,wdt:P2142,792910554,2.0,INCORRECT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
301,61,9QT,Is this triple correct or incorrect?,$0.50,AALKMII97,Submitted,240,98%,wd:Q1288004,wdt:P1412,wd:Q13330,2.0,INCORRECT,Object,Q1860
303,61,9QT,Is this triple correct or incorrect?,$0.50,GGUI83657S,Submitted,120,85%,wd:Q1288004,wdt:P1412,wd:Q13330,1.0,CORRECT,NaN,NaN
302,61,9QT,Is this triple correct or incorrect?,$0.50,HHCKW1111,Submitted,200,80%,wd:Q1288004,wdt:P1412,wd:Q13330,1.0,CORRECT,NaN,NaN


In [10]:
crowd_df.drop(columns=["Title", "Reward", "AssignmentStatus"], inplace=True)

In [11]:
crowd_df['LifetimeApprovalRate'] = crowd_df['LifetimeApprovalRate'].astype(str).str.rstrip('%').astype('float') / 100
crowd_df['WorkTimeInSeconds'] = crowd_df['WorkTimeInSeconds'].astype(float)

In [12]:
crowd_df

,HITId,HITTypeId,WorkerId,WorkTimeInSeconds,LifetimeApprovalRate,Input1ID,Input2ID,Input3ID,AnswerID,AnswerLabel,FixPosition,FixValue
AssignmentId,,,,,,,,,,,,
4,1,7QT,1726JMZQW,80.0,0.70,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
1,1,7QT,2133ICYWE97,60.0,0.99,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
2,1,7QT,2133U7HKDLO,40.0,0.40,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,yes,yes
5,1,7QT,2134U7HKDMM,2.0,0.70,wd:Q11621,wdt:P2142,792910554,1.0,CORRECT,NaN,NaN
3,1,7QT,928UJANWZ12,50.0,0.98,wd:Q11621,wdt:P2142,792910554,2.0,INCORRECT,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
301,61,9QT,AALKMII97,240.0,0.98,wd:Q1288004,wdt:P1412,wd:Q13330,2.0,INCORRECT,Object,Q1860
303,61,9QT,GGUI83657S,120.0,0.85,wd:Q1288004,wdt:P1412,wd:Q13330,1.0,CORRECT,NaN,NaN
302,61,9QT,HHCKW1111,200.0,0.80,wd:Q1288004,wdt:P1412,wd:Q13330,1.0,CORRECT,NaN,NaN


In [13]:
crowd_data = crowd_df[crowd_df['LifetimeApprovalRate'] >= 0.5]

In [14]:
average_work_time = crowd_data.groupby('WorkerId')['WorkTimeInSeconds'].mean()
workers_to_remove = average_work_time[average_work_time < 35].index
crowd_data = crowd_data[~crowd_data['WorkerId'].isin(workers_to_remove)]

In [15]:
tasks_solved_quickly = crowd_data[crowd_data['WorkTimeInSeconds'] < 10]

In [17]:
len(tasks_solved_quickly)

0

In [18]:
crowd_df.drop(columns=["WorkTimeInSeconds", "LifetimeApprovalRate"], inplace=True)

In [20]:
import numpy as np

json_files = ["./exports/people_db.json", "./exports/movie_db.json"]
merged_data = {
    k.split("/")[-1]: v
    for file_path in json_files
    for k, v in json.load(open(file_path, 'r')).items()
}

with open("./exports/predicate_db.json", 'r') as f:
    # Reversed index
    merged_data.update({v.split("/")[-1]: k for k, v in json.load(f).items()})


db = pd.read_pickle("./exports/extended_graph_triples.pkl")
db['subject_id'] = db['subject_id'].astype(str).str.strip()
db['subject_id'] = db['subject_id'].apply(lambda x: x.split("/")[-1])
db['predicate_label'] = db['predicate_label'].astype(str).str.strip()
db['object_label'] = db['object_label'].astype(str).str.strip()


def get_subject_label(subject_id):
    result = db[db['subject_id'] == subject_id]
    if not result.empty:
        return result.iloc[0]['subject_label']
    return subject_id

def set_label(value, required=True):
    value = str(value).strip()
    if "/" in value:
        value = value.split("/")[-1]
    elif ":" in value:
        value = value.split(":")[-1]
        if not required and value not in merged_data:
            value = get_subject_label(value)
    else:
        value = get_subject_label(value)

    return merged_data.get(value, np.nan if required else value)

for column in ["Input1ID", "Input2ID"]:
    crowd_df[column] = crowd_df[column].apply(set_label, required=True)

for column in ["Input3ID", "FixValue"]:
    crowd_df[column] = crowd_df[column].apply(set_label, required=False)


crowd_df = crowd_df.dropna(subset=["Input1ID", "Input2ID", "Input3ID"])

crowd_df


,HITId,HITTypeId,WorkerId,Input1ID,Input2ID,Input3ID,AnswerID,AnswerLabel,FixPosition,FixValue
AssignmentId,,,,,,,,,,
4,1,7QT,1726JMZQW,et the extraterrestrial,box office,792910554,1.0,CORRECT,NaN,nan
1,1,7QT,2133ICYWE97,et the extraterrestrial,box office,792910554,1.0,CORRECT,NaN,nan
2,1,7QT,2133U7HKDLO,et the extraterrestrial,box office,792910554,1.0,CORRECT,yes,yes
5,1,7QT,2134U7HKDMM,et the extraterrestrial,box office,792910554,1.0,CORRECT,NaN,nan
3,1,7QT,928UJANWZ12,et the extraterrestrial,box office,792910554,2.0,INCORRECT,NaN,nan
...,...,...,...,...,...,...,...,...,...,...
301,61,9QT,AALKMII97,nat pendleton,languages spoken written or signed,mirandese,2.0,INCORRECT,Object,english
303,61,9QT,GGUI83657S,nat pendleton,languages spoken written or signed,mirandese,1.0,CORRECT,NaN,nan
302,61,9QT,HHCKW1111,nat pendleton,languages spoken written or signed,mirandese,1.0,CORRECT,NaN,nan


In [22]:
result = (
    crowd_df.groupby(["Input1ID", "Input2ID", "Input3ID", "HITTypeId", "HITId"])
    .agg(
        Votes_CORRECT=("AnswerLabel", lambda x: (x == "CORRECT").sum()),
        Votes_INCORRECT=("AnswerLabel", lambda x: (x == "INCORRECT").sum()),
        Corrections_Subject=("FixValue", lambda x: x[crowd_df["FixPosition"] == "Subject"].tolist()),
        Corrections_Predicate=("FixValue", lambda x: x[crowd_df["FixPosition"] == "Predicate"].tolist()),
        Corrections_Object=("FixValue", lambda x: x[crowd_df["FixPosition"] == "Object"].tolist()),
    )
    .reset_index()
)

def ensure_single_value(corrections):
    unique_values = list(set([v for v in corrections if pd.notna(v) and v != "nan"]))
    if len(unique_values) > 1:
        raise ValueError(f"Multiple values found where a single value is expected: {unique_values}")
    return unique_values[0] if unique_values else np.nan

result["Corrections_Subject"] = result["Corrections_Subject"].apply(ensure_single_value)
result["Corrections_Predicate"] = result["Corrections_Predicate"].apply(ensure_single_value)
result["Corrections_Object"] = result["Corrections_Object"].apply(ensure_single_value)


def handle_nan_subject(row):
    """ If subject is wrong, then it is easier to swap the statement and correct the object """
    if not pd.isna(row["Corrections_Subject"]):
        row["Input1ID"], row["Input3ID"] = row["Input3ID"], row["Input1ID"]
        row["Corrections_Object"] = row["Corrections_Subject"]
        row["Corrections_Subject"] = np.nan
    return row

result = result.apply(handle_nan_subject, axis=1)

if not result["Corrections_Subject"].isna().all():
    raise Exception("Not all corrections have been handled")

result = result.drop(columns=["Corrections_Subject"])

result

,Input1ID,Input2ID,Input3ID,HITTypeId,HITId,Votes_CORRECT,Votes_INCORRECT,Corrections_Predicate,Corrections_Object
0,a happy event,cast member,theis schmidt,9QT,44,0,5,NaN,erika sainte
1,a night at the opera,cast member,jude law,9QT,58,1,4,NaN,rolfe sedan
2,william hurt,cast member,behind the candelabra,8QT,32,2,3,NaN,the blue butterfly
3,rasmus heise,director of photography,being with juli ashton,8QT,39,4,1,NaN,i kill giants
4,frederick worlock,cast member,bloody mama,8QT,38,2,3,NaN,airport
5,cho geunhyeon,country of citizenship,angola,9QT,60,2,3,NaN,south korea
6,conan the barbarian,voice actor,rose mcgowan,8QT,41,2,3,cast member,NaN
7,eskimo,production designer,john lee mahin,9QT,43,1,4,screenwriter,NaN
8,et the extraterrestrial,box office,792910554,7QT,1,4,1,NaN,NaN
9,finding nemo,box office,969023261,7QT,4,2,3,NaN,NaN


In [32]:
import numpy as np
import matplotlib.pyplot as plt
from statsmodels.stats.inter_rater import fleiss_kappa
import re
import unicodedata
import pandas as pd
import json


contingency_tables_latest = result.groupby(['HITTypeId', 'HITId'])[['Votes_CORRECT', 'Votes_INCORRECT']].sum()

fleiss_kappa_per_batch = contingency_tables_latest.groupby(level=0).apply(fleiss_kappa)

kappa_mapping = fleiss_kappa_per_batch.to_dict()

result['HITTypeId'] = result['HITTypeId'].map(kappa_mapping)

result['HITTypeId'] = result['HITTypeId'].map(lambda x: round(x, 3))

result

,Input1ID,Input2ID,Input3ID,HITTypeId,HITId,Votes_CORRECT,Votes_INCORRECT,Corrections_Predicate,Corrections_Object
0,a happy event,cast member,theis schmidt,-0.013,44,0,5,cast member,erika sainte
1,a night at the opera,cast member,jude law,-0.013,58,1,4,cast member,rolfe sedan
2,william hurt,cast member,behind the candelabra,0.152,32,2,3,cast member,the blue butterfly
3,rasmus heise,director of photography,being with juli ashton,0.152,39,4,1,director of photography,i kill giants
4,frederick worlock,cast member,bloody mama,0.152,38,2,3,cast member,airport
5,cho geunhyeon,country of citizenship,angola,-0.013,60,2,3,country of citizenship,south korea
6,conan the barbarian,voice actor,rose mcgowan,0.152,41,2,3,cast member,NaN
7,eskimo,production designer,john lee mahin,-0.013,43,1,4,screenwriter,NaN
8,et the extraterrestrial,box office,792910554,-0.108,1,4,1,box office,NaN
9,finding nemo,box office,969023261,-0.108,4,2,3,box office,NaN


In [71]:
################
# ADJUSTMENT
################
import warnings
warnings.filterwarnings("ignore")

result.replace("NaN", np.nan, inplace=True)
result["Corrections_Predicate"].fillna(result["Input2ID"], inplace=True)
result["Corrections_Object"].fillna(result["Input3ID"], inplace=True)
filtered_result = result.dropna(subset=["Corrections_Predicate", "Corrections_Object"], how='all')

# remove all where Vote_CORRECT > Vote_INCORRECT
filtered_result = filtered_result[filtered_result["Votes_CORRECT"]-2 < filtered_result["Votes_INCORRECT"]]

filtered_result

,Input1ID,Input2ID,Input3ID,HITTypeId,HITId,Votes_CORRECT,Votes_INCORRECT,Corrections_Predicate,Corrections_Object
0,a happy event,cast member,theis schmidt,-0.013,44,0,5,cast member,erika sainte
1,a night at the opera,cast member,jude law,-0.013,58,1,4,cast member,rolfe sedan
2,william hurt,cast member,behind the candelabra,0.152,32,2,3,cast member,the blue butterfly
4,frederick worlock,cast member,bloody mama,0.152,38,2,3,cast member,airport
5,cho geunhyeon,country of citizenship,angola,-0.013,60,2,3,country of citizenship,south korea
6,conan the barbarian,voice actor,rose mcgowan,0.152,41,2,3,cast member,rose mcgowan
7,eskimo,production designer,john lee mahin,-0.013,43,1,4,screenwriter,john lee mahin
9,finding nemo,box office,969023261,-0.108,4,2,3,box office,969023261
10,free willy 3 the rescue,screenwriter,angel de andres miquel,-0.013,45,2,3,screenwriter,keith walker
11,united states of america,country of origin,gandhi,0.152,35,3,2,country of origin,king solomons mines


In [72]:
def normalize_string(s):
    if not isinstance(s, str):
        return s
    normalized = unicodedata.normalize('NFKD', s.lower())
    cleaned = re.sub(r'[^\w\s]', '', normalized)  # Remove punctuation
    return ' '.join(cleaned.split())  # Remove redundant spaces


def create_correction_index(df):
    index = {}

    for _, row in df.iterrows():
        input1_id = row["Input1ID"]
        if input1_id not in index:
            index[input1_id] = []

        for column in ["Input1ID", "Input2ID", "Input3ID", "Corrections_Predicate", "Corrections_Object"]:
            row[column] = normalize_string(row[column]) if row["Input2ID"] != "publication date" and row["Corrections_Predicate"] != "publication date" else row[column]

        if pd.notna(row["Corrections_Predicate"]) and row["Corrections_Predicate"]:
            index[input1_id].append((
                row["Input2ID"],            # Predicate
                row["Input3ID"],            # Object
                row["Corrections_Predicate"],  # Corrected Predicate
                row["Input3ID"],            # Corrected Object
                row["Votes_CORRECT"],
                row["Votes_INCORRECT"],
                row["HITTypeId"]
            ))

    return index


correction_index = create_correction_index(filtered_result)

In [73]:
correction_index

{'a happy event': [('cast member',
   'theis schmidt',
   'cast member',
   'theis schmidt',
   0,
   5,
   -0.013)],
 'a night at the opera': [('cast member',
   'jude law',
   'cast member',
   'jude law',
   1,
   4,
   -0.013)],
 'william hurt': [('cast member',
   'behind the candelabra',
   'cast member',
   'behind the candelabra',
   2,
   3,
   0.152)],
 'frederick worlock': [('cast member',
   'bloody mama',
   'cast member',
   'bloody mama',
   2,
   3,
   0.152)],
 'cho geunhyeon': [('country of citizenship',
   'angola',
   'country of citizenship',
   'angola',
   2,
   3,
   -0.013)],
 'conan the barbarian': [('voice actor',
   'rose mcgowan',
   'cast member',
   'rose mcgowan',
   2,
   3,
   0.152)],
 'eskimo': [('production designer',
   'john lee mahin',
   'screenwriter',
   'john lee mahin',
   1,
   4,
   -0.013)],
 'finding nemo': [('box office',
   '969023261',
   'box office',
   '969023261',
   2,
   3,
   -0.108)],
 'free willy 3 the rescue': [('screenwrite

In [74]:
def generate_question(predicate, obj, entity):
    templates = {
        'box office': "What is the box office of {entity}?",
        'publication date': "What is the publication date of {entity}?",
        'director of photography': "Who is the director of photography of {obj}?",
        'country of origin': "What is the country of origin of {obj}?",
        'executive producer': "Who is the executive producer of {entity}?",
        'art director': "Who is the art director of {entity}?",
        'cast member': "Who is the cast member {entity} in {obj}?",
        'production company': "Who is the production company of {entity}?",
        'place of birth': "What is the place of birth of {entity}?",
        'distributed by': "Who distributed {entity}?",
        'country of citizenship': "What is the country of citizenship of {entity}?",
        'original language of film or tv show': "What is the original language of the film {entity}?",
    }
    template = templates.get(predicate, "What is the {predicate} of {entity}?")
    return template.format(entity=entity.title(), obj=obj.title(), predicate=predicate)

def generate_answer(predicate, obj, entity):
    templates = {
        'box office': "The box office of {entity} is {object}.",
        'publication date': "The publication date of {entity} is {object}.",
        'director of photography': "The director of photography of {entity} is {object}.",
        'country of origin': "The country of origin of {entity} is {object}.",
        'executive producer': "The executive producer of {entity} is {object}.",
        'art director': "The art director of {entity} is {object}.",
        'cast member': "{object} is a cast member of {entity}.",
        'production company': "The production company of {entity} is {object}.",
        'place of birth': "The place of birth of {entity} is {object}.",
        'distributed by': "{entity} was distributed by {object}.",
        'country of citizenship': "The country of citizenship of {entity} is {object}.",
        'original language of film or tv show': "The original language of {entity} is {object}.",
    }
    template = templates.get(predicate, "{object}")
    return template.format(entity=entity.title(), object=obj.title(), predicate=predicate)

def generate_qa_pairs(index):
    qa_pairs = []
    for entity, corrections in index.items():
        for correction in corrections:
            orig_predicate, orig_object, corrected_predicate, corrected_object, votes_correct, votes_incorrect, inter_agreement = correction

            predicate = corrected_predicate if corrected_predicate else orig_predicate
            obj = corrected_object if corrected_object else orig_object

            question = generate_question(predicate, obj, entity)
            answer = generate_answer(predicate, obj, entity)
            answer += f"\n[Crowd, inter-rater agreement {inter_agreement}, The answer distribution for this specific task was {votes_correct} support votes, {votes_incorrect} reject votes]"

            qa_pairs.append({'question': question, 'answer': answer})
    return qa_pairs


In [75]:
qa_pairs = generate_qa_pairs(correction_index)

In [76]:
qa_pairs

[{'question': 'Who is the cast member A Happy Event in Theis Schmidt?',
  'answer': 'Theis Schmidt is a cast member of A Happy Event.\n[Crowd, inter-rater agreement -0.013, The answer distribution for this specific task was 0 support votes, 5 reject votes]'},
 {'question': 'Who is the cast member A Night At The Opera in Jude Law?',
  'answer': 'Jude Law is a cast member of A Night At The Opera.\n[Crowd, inter-rater agreement -0.013, The answer distribution for this specific task was 1 support votes, 4 reject votes]'},
 {'question': 'Who is the cast member William Hurt in Behind The Candelabra?',
  'answer': 'Behind The Candelabra is a cast member of William Hurt.\n[Crowd, inter-rater agreement 0.152, The answer distribution for this specific task was 2 support votes, 3 reject votes]'},
 {'question': 'Who is the cast member Frederick Worlock in Bloody Mama?',
  'answer': 'Bloody Mama is a cast member of Frederick Worlock.\n[Crowd, inter-rater agreement 0.152, The answer distribution for

In [77]:
qa_df = pd.DataFrame(qa_pairs)

In [78]:
qa_df

,question,answer
0,Who is the cast member A Happy Event in Theis ...,Theis Schmidt is a cast member of A Happy Even...
1,Who is the cast member A Night At The Opera in...,Jude Law is a cast member of A Night At The Op...
2,Who is the cast member William Hurt in Behind ...,Behind The Candelabra is a cast member of Will...
3,Who is the cast member Frederick Worlock in Bl...,Bloody Mama is a cast member of Frederick Worl...
4,What is the country of citizenship of Cho Geun...,The country of citizenship of Cho Geunhyeon is...
5,Who is the cast member Conan The Barbarian in ...,Rose Mcgowan is a cast member of Conan The Bar...
6,What is the screenwriter of Eskimo?,"John Lee Mahin\n[Crowd, inter-rater agreement ..."
7,What is the box office of Finding Nemo?,The box office of Finding Nemo is 969023261.\n...
8,What is the screenwriter of Free Willy 3 The R...,"Angel De Andres Miquel\n[Crowd, inter-rater ag..."
9,What is the country of origin of Gandhi?,The country of origin of United States Of Amer...


In [79]:
qa_df.to_csv("crowdsourcer_precomputed.csv")

In [ ]:
def find_crowdsourced_answer(query):
    # Embed the user query
    query_embedding = qe.embed_phrase(query)

    # Retrieve similar questions
    similar_questions = self.retrieve_similar_questions(query_embedding)

    # Check if similarity is above threshold
    for sim_question, sim_score in similar_questions:
        if sim_score > self.similarity_threshold:
            # Return the corresponding answer
            answer = self.qa_pairs[sim_question]
            return answer

    return None

In [39]:
import numpy as np
from sentence_transformers import SentenceTransformer
import torch
from utils.utils import get_device
import logging

logging.getLogger("torch").setLevel(logging.WARNING)

class QueryEmbedderContextualized:
    def __init__(self, model_name='sentence-transformers/all-mpnet-base-v2'):
        """Initializes the QueryEmbedder with a SentenceTransformer model and device setup."""
        self.device = get_device()
        self.model = SentenceTransformer(model_name, device=self.device)
        self.cache = {}

    def embed_phrase(self, phrases):
        """
        Generates embeddings for given phrases using SentenceTransformer, with caching.

        Args:
            phrases (str or List[str]): Input phrase(s) to embed.

        Returns:
            np.ndarray: Embedding vector(s) for the input phrase(s).
        """
        if isinstance(phrases, str):
            phrases = [phrases]
        elif not isinstance(phrases, list):
            raise TypeError("Input must be a string or a list of strings.")

        phrases_to_compute = [p for p in phrases if p not in self.cache]
        cached_embeddings = [self.cache[p] for p in phrases if p in self.cache]

        if phrases_to_compute:
            new_embeddings = self.model.encode(
                phrases_to_compute,
                show_progress_bar=False,
                convert_to_numpy=True,
                normalize_embeddings=True
            )

            for phrase, emb in zip(phrases_to_compute, new_embeddings):
                self.cache[phrase] = emb
            cached_embeddings.extend(new_embeddings)

        return cached_embeddings[0] if len(cached_embeddings) == 1 else np.array(cached_embeddings)


In [40]:
qe = QueryEmbedderContextualized()

In [56]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd

model = SentenceTransformer('all-MiniLM-L6-v2')

crowdsourcing_questions = [
    "Who is the director of photography of Being With Juli Ashton?",
    "Who is the director of photography of I Kill Giants?",
    "What is the box office of Et The Extraterrestrial?",
    "What is the country of origin of Gandhi?",
    "What is the country of origin of King Solomons Mines?",
    "What is the publication date of God Help The Girl?",
    "What is the box office of Kung Fu Panda 3?",
    "What is the country of citizenship of Masaichi Nagata?",
    "Who distributed Medea?",
    "What is the place of birth of Miranda Frigon?",
    "Who is the production company of Mulan?",
    "Who is the art director of Naruto The Movie Guardians Of The Crescent Moon Kingdom?",
    "What is the original language of the film English?",
    "What is the box office of Peaceful Warrior?",
    "What is the box office of Robocop 3?",
    "What is the publication date of Scent Of A Woman?",
    "What is the publication date of Sex Death And Bowling?",
    "What is the publication date of The Browning Version?",
    "Who is the cast member The Hell Of 63 in Giuseppe Rinaldi?",
    "What is the publication date of The Partys Just Beginning?",
    "What is the box office of The Princess And The Frog?",
    "Who is the executive producer of Xmen First Class?"
]

crowdsourcing_embeddings = model.encode(crowdsourcing_questions, convert_to_tensor=True)

def classify_query(query):

    query_embedding = model.encode(query, convert_to_tensor=True)
    similarities = util.pytorch_cos_sim(query_embedding, crowdsourcing_embeddings)
    
    max_similarity = similarities.max().item()
    print(max_similarity)
    if max_similarity > 0.9:
        return "crowdsourcing"
    else:
        return "factual"


In [58]:
test_queries = [
    "Who is the director of photography of I Kill Giants?",
    "What is the box office of The Princess And The Frog?",
    "Who distributed Medea?",
    "Who is the production company of Mulan?",
    "Who is the producer of xmen?",
    "Where was Angelina Jolie born?"
]

for query in test_queries:
    label = classify_query(query)
    print(f"Query: {query} => Classified as: {label}")

1.0
Query: Who is the director of photography of I Kill Giants? => Classified as: crowdsourcing
1.0
Query: What is the box office of The Princess And The Frog? => Classified as: crowdsourcing
1.0
Query: Who distributed Medea? => Classified as: crowdsourcing
1.0
Query: Who is the production company of Mulan? => Classified as: crowdsourcing
0.8891483545303345
Query: Who is the producer of xmen? => Classified as: factual
0.4420839250087738
Query: Where was Angelina Jolie born? => Classified as: factual
